In [3]:
import gradio as gr
import sys
import logging
from dotenv import load_dotenv

# Reset và cấu hình logging để hiển thị logs từ chatbot_main.py
# Xóa tất cả handlers cũ
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Cấu hình lại logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ],
    force=True  # Force reconfiguration
)

load_dotenv()
sys.path.append('../backend')
from chatbot_main import chatbot_response

2025-11-26 09:21:25,665 - INFO - HTTP Request: GET https://2f41421b-1095-4cb3-8625-6df79db78d48.us-east-1-1.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-11-26 09:21:29,548 - INFO - HTTP Request: GET https://2f41421b-1095-4cb3-8625-6df79db78d48.us-east-1-1.aws.cloud.qdrant.io:6333/collections/embedding_data "HTTP/1.1 200 OK"


In [4]:
def chat_interface(message, history):
    """
    Hàm xử lý chat với lịch sử hội thoại
    
    Args:
        message: Tin nhắn từ người dùng
        history: Lịch sử hội thoại (list of tuples)
    
    Returns:
        Phản hồi từ chatbot
    """
    try:
        # Gọi chatbot backend (logging sẽ tự động in ra từ chatbot_main.py)
        response = chatbot_response.invoke(message)
        return response
    except Exception as e:
        logging.error(f"Error occurred: {str(e)}", exc_info=True)
        return f"Xin lỗi, đã xảy ra lỗi: {str(e)}"

In [5]:
# Tạo giao diện Gradio với ChatInterface
demo = gr.ChatInterface(
    fn=chat_interface,
    title="Chatbot Y tế",
    description="",
    examples=[
        "Thuốc Paracetamol có tác dụng gì?",
        "Cách sử dụng thuốc giảm đau an toàn?",
        "Tôi bị đau đầu nên uống thuốc gì?",
        "Thuốc kháng sinh có tác dụng phụ gì?",
        "Cách bảo quản thuốc đúng cách?"
    ],
    theme=gr.themes.Default(
        primary_hue="blue",
        secondary_hue="sky",
        neutral_hue="slate",
        font=["Inter", "Segoe UI", "Roboto", "Helvetica Neue", "Arial", "sans-serif"],
        font_mono=["Fira Code", "Consolas", "Monaco", "monospace"]
    ).set(
        body_background_fill="white",
        body_text_color="#1f2937",
        body_text_size="16px",
        button_primary_background_fill="#3b82f6",
        button_primary_text_color="white",
        block_title_text_color="#111827",
        block_title_text_size="20px",
        block_title_text_weight="600",
        block_label_text_color="#374151",
        block_label_text_size="15px",
        block_label_text_weight="500",
        input_background_fill="white",
        input_text_size="16px",
    ),
    css="""
    .message {
        font-size: 16px !important;
        line-height: 1.6 !important;
    }


    h1 {
        font-weight: 700 !important;
        letter-spacing: -0.02em !important;
        text-align: center !important;
    }
    .gradio-container p {
        text-align: center !important;
    }
    """
)


/Users/lebaminhphuc/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


2025-11-26 09:21:32,984 - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [ ]:
# Khởi chạy ứng dụng
if __name__ == "__main__":
    demo.launch(
        share=False,  # Đặt True nếu muốn chia sẻ link public
        show_error=True
    )


* Running on local URL:  http://127.0.0.1:7860
2025-11-26 09:21:31,909 - INFO - HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2025-11-26 09:21:31,918 - INFO - HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
* To create a public link, set `share=True` in `launch()`.


2025-11-26 09:21:35,035 - INFO - Cap nhat long term memory 


2025-11-26 09:21:35,036 - INFO - Lay data long term memory 


2025-11-26 09:21:35,036 - INFO - Mo rong query 


2025-11-26 09:21:37,550 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-26 09:21:39,456 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-26 09:21:39,475 - INFO - Lay document phu hop 


2025-11-26 09:21:42,794 - INFO - HTTP Request: POST https://2f41421b-1095-4cb3-8625-6df79db78d48.us-east-1-1.aws.cloud.qdrant.io:6333/collections/embedding_data/points/query "HTTP/1.1 200 OK"
2025-11-26 09:21:43,867 - INFO - HTTP Request: POST https://2f41421b-1095-4cb3-8625-6df79db78d48.us-east-1-1.aws.cloud.qdrant.io:6333/collections/embedding_data/points/query "HTTP/1.1 200 OK"
2025-11-26 09:21:43,870 - INFO - Kiem tra do dai document
2025-11-26 09:21:43,871 - INFO - Tra loi nguoi dung
2025-11-26 09:21:48,612 - INFO - HTTP Request: PO